In [3]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

In [1]:
%pip install openai-agents python-dotenv -qU


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Basic Agent Creation

In [7]:
# Agent creation 
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions = "You are a helpful assistant.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, 'Hi, How are you?')
result.final_output

"I'm here and ready to help! How about you?"

In [11]:
# Agent 2 example 

joke_agent = Agent(
    name = "Joke Agent",
    instructions = "You are a funny assistant. Your jobs is to tell a funny joke.",
    model = "gpt-4o-mini"
)

joke_agent_result = await Runner.run(joke_agent, 'Tell me a joke')
joke_agent_result.final_output

"Why don't skeletons fight each other? \n\nThey don't have the guts!"

In [15]:
# Agent 3 example

language_translator_agent = Agent(
    name = "Language Translator Agent",
    instructions = "You are a translator. Your job is to translate given text into urdu.",
    model = "gpt-4o-mini"
)

language_translator_agent_result = await Runner.run(language_translator_agent, f"translate this joke: {joke_agent_result.final_output} into urdu")
print(language_translator_agent_result.final_output)

ہڈیاں ایک دوسرے سے کیوں نہیں لڑتیں؟

کیونکہ ان میں حوصلہ نہیں ہوتا!


# Structured output using Pydantic

In [19]:
# Structured output example
from pydantic import BaseModel

class Recipe(BaseModel):
    name: str
    ingredients: list[str]
    instructions: str
    cooking_time: int
    
recipe_agent = Agent(
    name = "Recipe Agent",
    instructions = "You are a recipe generator. Your job is to generate a recipe.",
    model = "gpt-4o-mini",
    output_type=Recipe
)
recipe_agent_result = await Runner.run(recipe_agent, "Generate a recipe for a biryani.")
print(recipe_agent_result.final_output)



name='Chicken Biryani' ingredients=['2 cups basmati rice', '500 grams chicken, cut into pieces', '1 large onion, thinly sliced', '2 tomatoes, chopped', '1/2 cup plain yogurt', '4 cups water', '4 tablespoons biryani masala', '2 bay leaves', '4 green cardamom pods', '5 cloves', '1 cinnamon stick', '4 tablespoons oil or ghee', 'Salt to taste', 'Fresh cilantro and mint for garnishing', 'Saffron strands (optional)'] instructions='1. Rinse the basmati rice under cold water until the water runs clear. Soak the rice in water for 30 minutes, then drain.  \n2. In a large pot, heat oil or ghee over medium heat. Add onions and sauté until golden brown.  \n3. Add the chicken pieces and cook until they turn white.  \n4. Stir in the chopped tomatoes and cook until soft.  \n5. Add yogurt and biryani masala, mixing well. Cook until the chicken is tender and the oil separates from the mixture.  \n6. In another pot, bring 4 cups of water to a boil. Add soaked and drained rice along with bay leaves, carda

# Function Tool in Agent

In [23]:
from agents import Agent, Runner, function_tool

@function_tool
def getTemperature(city: str) -> str:
    return f"70 Degree Fahrenheit in {city}"

@function_tool
def get_weather(city: str) -> str:
    return f"Sunny in {city}"


weather_agent = Agent(
    name = "Weather Agent",
    instructions = "You are a weather agent. Your job is to get the weather of a city and temperature.",
    model = "gpt-4o-mini",
    tools=[getTemperature, get_weather]
)
weather_agent_result = await Runner.run(weather_agent, "Get the weather and current temperature of Karachi.")
print(weather_agent_result.final_output)

The weather in Karachi is sunny, with a current temperature of 70°F.


# Handoffs 
Handoffs allow an agent to delegate tasks to another agent
This is particularly useful in scenarios where different agents specialize in distinct areas. For example, a customer support app might have agents that each specifically handle tasks like order status, refunds, FAQs, etc.

In [24]:
from agents import Agent, Runner, function_tool
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str

Tutorial_agent = Agent(
    name = "Tutorial Agent",
     handoff_description = "use for generating tutorials based on given outline and topic",
    instructions = ("You are a tutorial agent. Your job is to create a tutorial for a given topic.", "give detail tutorial on every topic of the outline also add comments explanation and code snippets."),
    model = "gpt-4o-mini",
    output_type = Tutorial
)


outline_agent = Agent(
    name = "Outline Agent",
    instructions = "You are an outline agent. Your job is to create an outline for a given topic.",
    model = "gpt-4o-mini",
    handoffs = [Tutorial_agent]
)

tutorial_response = await Runner.run(outline_agent, "Create an outline for a tutorial on Python lists.")
print(tutorial_response.final_output)


# Outline for a Tutorial on Python Lists

## I. Introduction
   A. What are Lists?
   B. Importance of Lists in Python
   C. Overview of Tutorial Structure

## II. Creating Lists
   A. Syntax for List Creation
   B. Different Ways to Create Lists
      1. Using Square Brackets
      2. Using the `list()` Function
      3. Creating an Empty List
   C. Example: Basic List Creation

## III. Accessing List Elements
   A. Indexing in Python Lists
      1. Positive Indexing
      2. Negative Indexing
   B. Slicing Lists
      1. Basic Slicing
      2. Advanced Slicing Techniques
   C. Example: Accessing Elements in a List

## IV. Modifying Lists
   A. Adding Elements
      1. Using `append()`
      2. Using `insert()`
      3. Using `extend()`
   B. Removing Elements
      1. Using `remove()`
      2. Using `pop()`
      3. Using `clear()`
   C. Example: Modifying List Contents

## V. List Methods and Functions
   A. Common List Methods
      1. `sort()`
      2. `reverse()`
      3. `count(

# Multi AI Agents using handoffs 

In [32]:
from agents import Agent, Runner, function_tool, handoff, RunContextWrapper


math_tutor_agent = Agent(
    name = "Math Tutor Agent",
    handoff_description = "Specialized agent for solving math problems",
    instructions = "You are a math tutor. Your job is to solve math problems.",
    model = "gpt-4o-mini",
)

def math_tutor_handoff(context: RunContextWrapper[None]):
    print('Handing off to math tutor agent')
    

history_tutor_agent = Agent(
    name = "History Tutor Agent",
    handoff_description = "Specialized agent for solving history problems",
    instructions = "You are a history tutor. Your job is to solve history problems.",
    model = "gpt-4o-mini",
)

def history_tutor_handoff(context: RunContextWrapper[None]):
    print('Handing off to history tutor agent')


multi_agent_team = Agent(
    name = "Multi-Agent Team",
    instructions = "You are a multi-agent team. Your job is to solve problems using specialized agents.",
    handoffs=[handoff(math_tutor_agent, on_handoff=math_tutor_handoff), handoff(history_tutor_agent, on_handoff=history_tutor_handoff)],
)

result = await Runner.run(multi_agent_team, "who is the first president of USA")
result.final_output

Handing off to history tutor agent


'The first president of the United States was George Washington. He served from April 30, 1789, to March 4, 1797. Washington is often called the "Father of His Country" for his pivotal role in leading the nation during the founding years.'

# Streaming 

## Raw response events

RawResponsesStreamEvent are raw events passed directly from the LLM. They are in OpenAI Responses API format, which means each event has a type (like response.created, response.output_text.delta, etc) and data. These events are useful if you want to stream response messages to the user as soon as they are generated.

In [39]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model="gpt-4o-mini",
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

# In a Jupyter notebook, use await instead of asyncio.run()
await main()

Sure! Here are five jokes for you:

1. **Why don't scientists trust atoms?**  
   Because they make up everything!

2. **What do you call fake spaghetti?**  
   An impasta!

3. **Why did the scarecrow win an award?**  
   Because he was outstanding in his field!

4. **How do you organize a space party?**  
   You planet!

5. **Why did the bicycle fall over?**  
   Because it was two-tired! 

Hope these made you smile!

# Guardrails 

Guardrails run in parallel to your agents, enabling you to do checks and validations of user input. For example, imagine you have an agent that uses a very smart (and hence slow/expensive) model to help with customer requests. You wouldn't want malicious users to ask the model to help them with their math homework. So, you can run a guardrail with a fast/cheap model. If the guardrail detects malicious usage, it can immediately raise an error, which stops the expensive model from running and saves you time/money.

There are two kinds of guardrails:

1. Input guardrails run on the initial user input
2. Output guardrails run on the final agent output